In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('/data_vault/hexai02/CarpalTunnel/AI Project Data Clean 10-8.xlsx')
df['Clinical Signs of CTS'].unique()
df['Clinical Signs of CTS'] = df['Clinical Signs of CTS'].replace('Yes', 'Y')
df['Clinical Signs of CTS'].unique()
count_question_mark = (df['Clinical Signs of CTS'] == '?').sum()
df = df[df['Clinical Signs of CTS'] != '?']
df['Number'] = df['Number'].astype(str)
# Replace both variations of 'Unreported/Chose not to disclose' with a unified value
df['Ethnicity'] = df['Ethnicity'].replace(['Unreported/Chose not to disclose', 'Unreported/Chose Not to Disclose'], 
                                          'Unreported/Chose not to disclose')
df = df.dropna(subset='Measurement at Wrist')
mean = df['Age'].mean()
df['Age'].fillna(mean, inplace=True)

/tmp/ipykernel_3832902/246703226.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(mean, inplace=True)


In [4]:
df.head()

,Number,WristImage Segmentation for AI Analysis,Arm (L/R),Sex,Race,Ethnicity,Age,Clinical Signs of CTS,CTS-6 Score,Measurement at Wrist
0,21,Yes,R,M,White,Non-Hispanic,43.0,Y,12.5,12.16
1,22,Yes,R,M,White,Unreported/Chose not to disclose,74.0,Y,16.5,20.10
2,23,Yes,L,M,White,Unreported/Chose not to disclose,74.0,Y,11.5,13.83
3,24,Yes,R,F,Black,Non-Hispanic,63.0,Y,21.0,16.55
4,25,Yes,R,M,White,Non-Hispanic,49.0,Y,9.0,19.21


In [5]:
df.isna().sum()

Number                                      0
WristImage Segmentation for AI Analysis     0
Arm (L/R)                                   0
Sex                                         0
Race                                        0
Ethnicity                                   0
Age                                         0
Clinical Signs of CTS                       0
CTS-6 Score                                16
Measurement at Wrist                        0
dtype: int64

In [6]:
test_df = df.dropna(subset=['Clinical Signs of CTS', 'CTS-6 Score'])

In [ ]:
restructured_root = 'data_yolo'
image_dir = os.path.join(restructured_root, 'images')
image_train_dir = os.path.join(image_dir, 'train')
image_val_dir = os.path.join(image_dir, 'val')
image_test_dir = os.path.join(image_dir, 'test')
label_dir = os.path.join(restructured_root, 'labels')
label_train_dir = os.path.join(label_dir, 'train')
label_val_dir = os.path.join(label_dir, 'val')

In [ ]:
anno_dir = '/data_vault/hexai02/CarpalTunnel/Annotations'
dicom_dir = '/data_vault/hexai02/CarpalTunnel/Images'

In [ ]:
def load_mask(mask_path):
    mask = sitk.ReadImage(mask_path)
    return sitk.GetArrayFromImage(mask)[0][: 450, 200: 1300]

def load_dicom(dicom_path):
    dicom_data = pydicom.dcmread(dicom_path)
    return dicom_data.pixel_array[: 450, 200: 1300]